# Consensus partitions calculation

Here representative module partitions are calculated for entire task and condition classes. First, agreement matrix $D$ between all partitions of interest is calculated. An element of agreement matrix $D_{ij}$ represents the number of times that nodes $i$ and $j$ are placed within the same module. Then, agreement matrix $D$ is normalized to reflect the probability that two nodes are a part of the same community (given random partition). Normalized agreemnent matrix is then thresholded at $\tau$ level, equal to `consensus_tau` (here 0.5), to remove weak associations. Resulting matrix is then reclustered using Louvain algorithm `consensus_reps` times. Analysis results in five representative partitions: 
- `all`: entire task partition
- `rew`: reward-seeking condition parition
- `pun`: punishment-avoiding condition partition
- `inc`: increasing (positive) prediciton error partition
- `dec`: decreasing (negative) prediction error partition

Partitions are stored under `consensus_tau_<consensus_tau>.json` file within:
```
└── <atlas_name>
    └── <threshold>
        ├── gamma_<louvain_gamma>
        │   ├── consensus_tau_0_5.json
        │   ├── consensus_tau_0_75.json
        │   └── ...
```
Output JSON file contains keys representing different partition classes and values corresponding to partition vector. It also contains other metadata like `consensus_tau` and `consensus_reps` values.

In [1]:
import json
from os.path import join

import numpy as np
import pandas as pd
from bct.algorithms.clustering import agreement, consensus_und
from dn_utils.path import path
from tqdm.notebook import tqdm

In [6]:
atlas = "combined_roi_4and5"
gamma_range = np.arange(0.5, 2, 0.5)

consensus_tau = 0.5
consensus_reps = 1_000

In [7]:
# Load graph measures
path_corrmats = join(path["bsc"], "corrmats")
path_corrmats_unthr = join(path_corrmats, atlas, "unthr")

m = {}
for gamma in gamma_range:
    gamma_str = str(float(gamma)).replace(".", "_")
    path_corrmats_unthr_gamma = join(path_corrmats_unthr, f"gamma_{gamma_str}")
    m[gamma] = np.load(join(path_corrmats_unthr_gamma, "m_aggregated.npy"))

# Load subject exclusion
df_exclusion = pd.read_csv(join(path["nistats"], "exclusion/exclusion.csv"), 
                           index_col=0)
ok_index = df_exclusion["ok_all"]

# Meta information about corrmats dimensions
with open(join(path_corrmats, atlas, "corrmats_aggregated.json"), "r") as f:
    corrmats_meta = json.loads(f.read()) 
    
n_subjects_ok = sum(ok_index)
n_conditions = len(corrmats_meta["dim2"])
n_perr_sign = len(corrmats_meta["dim3"])
n_roi = len(corrmats_meta["dim4"])

In [53]:
for gamma in gamma_range:

    print(f"γ = {gamma}")
    
    consensus = {}
    
    mt = m[gamma][ok_index]
    m_all = np.reshape(mt, (n_subjects_ok * n_conditions * n_perr_sign, n_roi))
    m_rew = np.reshape(mt[:, 0], (n_subjects_ok * n_perr_sign, n_roi))
    m_pun = np.reshape(mt[:, 1], (n_subjects_ok * n_perr_sign, n_roi))
    m_inc = np.reshape(mt[:, :, 0], (n_subjects_ok * n_perr_sign, n_roi))
    m_dec = np.reshape(mt[:, :, 1], (n_subjects_ok * n_perr_sign, n_roi))

    for k, v in tqdm([("all", m_all), ("rew", m_rew), ("pun", m_pun), 
                      ("inc", m_inc), ("dec", m_dec)]):

        D = agreement(v.T) / len(v)
        consensus_array = consensus_und(
            D, tau=consensus_tau, reps=consensus_reps)
        
        # Coerce to int so it is JSON serializable
        consensus[k] = [int(i) for i in consensus_array]
    
    # Save as JSON file
    consensus["consensus_reps"] = consensus_reps
    gamma_str = str(float(gamma)).replace(".", "_")
    tau_str = str(float(consensus_tau)).replace(".", "_")    
    path_corrmats_unthr_gamma = join(path_corrmats_unthr, f"gamma_{gamma_str}")
    path_json = join(path_corrmats_unthr_gamma, f"consensus_tau_{tau_str}.json")
        
    with open(path_json, "w") as f:
        f.write(json.dumps(consensus, indent=2))

γ = 0.5


  0%|          | 0/5 [00:00<?, ?it/s]

γ = 1.0


  0%|          | 0/5 [00:00<?, ?it/s]

γ = 1.5


  0%|          | 0/5 [00:00<?, ?it/s]